<a href="https://colab.research.google.com/github/romy-vos/Big-Data-Optimization/blob/master/Data_Aggregation_15_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/img/logo_amds.png?raw=1" alt="Logo" width=128px/>

# VUmc Research Project - Reinforcement Learning for Sepsis Prevention
# Data Aggregation

AmsterdamUMCdb version 1.0.2 March 2020  
Copyright &copy; 2003-2022 Amsterdam UMC - Amsterdam Medical Data Science

## 1. Set up the environment variables for Colab and GoogleBigQuery to access

In [1]:
import os
from google.colab import auth
from IPython.display import display
from google.colab import drive
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from IPython.display import display
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('use_inf_as_na', True)

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/MLRFH')
 
#sets dateset
PROJECT_ID = 'rl-aki'
DATASET_ID = 'version1_0_2'
LOCATION = 'eu'
 
#all libraries check this environment variable, so set it:
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
 
auth.authenticate_user()
print('Authenticated')

Mounted at /content/drive
Authenticated


## 2. Define Preprocessing Functions

In [45]:
#Some preprocessing functions 

def to_cols(data):

  grouped = data.pivot_table(index=['admissionid', 'time'], 
          columns=['item'], values='value')

  return grouped
  

def to_cols_action(data):

  grouped = data.pivot_table(index=['admissionid', 'time'], 
            columns=['item'], values='administered')

  return grouped

def remove_outliers(data):
  #delete outliers
  data = data.reset_index() #return to single index

  #select outlier cols
  all_cols = ['Kreatinine', 'Kreatinine (bloed)', 'KREAT enzym. (bloed)',
       'UrineSupraPubis', 'UrineIncontinentie', 'Nefrodrain re Uit',
       'Nefrodrain li Uit', 'UrineSpontaan', 'UrineUP', 'UrineSplint Re',
       'UrineSplint Li', 'UrineCAD', 'Chloor (bloed)', 'Natrium (bloed)',
       'Kalium (bloed)', 'HCO3', 'Natrium', 'Natrium Astrup',
       'Kalium Astrup', 'Chloor Astrup', 'Chloor', 'Kalium',
       'Act.HCO3 (bloed)', 'Na (onv.ISE) (bloed)', 'K (onv.ISE) (bloed)',
       'Cl (onv.ISE) (bloed)', 'Niet invasieve bloeddruk gemiddeld',
       'ABP gemiddeld II', 'ABP gemiddeld']
  
  # Natrium
  data['Natrium'][(data['Natrium'] < 65.) & (data['Natrium'] > 165.)] = np.nan
  data['Natrium (bloed)'][(data['Natrium (bloed)'] < 65.) & (data['Natrium (bloed)'] > 165.)] = np.nan
  data['Natrium Astrup'][(data['Natrium Astrup'] < 65.) & (data['Natrium Astrup'] > 165.)] = np.nan
  data['Na (onv.ISE) (bloed)'][(data['Na (onv.ISE) (bloed)'] < 65.) & (data['Na (onv.ISE) (bloed)'] > 165.)] = np.nan
  
  # Mean Blood Pressure
  data['ABP gemiddeld'][(data['ABP gemiddeld'] < 30.) & (data['ABP gemiddeld'] > 165.)] = np.nan
  data['Niet invasieve bloeddruk gemiddeld'][(data['Niet invasieve bloeddruk gemiddeld'] < 30.) & (data['Niet invasieve bloeddruk gemiddeld'] > 165.)] = np.nan
  data['ABP gemiddeld II'][(data['ABP gemiddeld II'] < 30) & (data['ABP gemiddeld II'] > 165)]

  # Kalium
  data['Kalium'][data['Kalium'] > 12.] = np.nan
  data['Kalium (bloed)'][data['Kalium (bloed)'] > 12.] = np.nan
  data['Kalium Astrup'][data['Kalium Astrup'] > 12.] = np.nan
  data['K (onv.ISE) (bloed)'][data['K (onv.ISE) (bloed)'] > 12.] = np.nan

  # Kreatinine
  data['Kreatinine'][data['Kreatinine'] < 30.] = np.nan
  data['Kreatinine (bloed)'][data['Kreatinine (bloed)'] < 30.] = np.nan
  data['KREAT enzym. (bloed)'][data['KREAT enzym. (bloed)'] < 30.] = np.nan

  # Bicarbonate
  data['HCO3'][(data['HCO3'] < 5.) & (data['HCO3'] > 45.)] = np.nan
  data['Act.HCO3 (bloed)'][(data['Act.HCO3 (bloed)'] < 5.) & (data['Act.HCO3 (bloed)'] > 45.)] = np.nan

  # Chloor
  data['Chloor'][(data['Chloor'] < 50.) & (data['Chloor'] > 200.)] = np.nan
  data['Chloor (bloed)'][(data['Chloor (bloed)'] < 50.) & (data['Chloor (bloed)'] > 200.)] = np.nan
  data['Chloor Astrup'][(data['Chloor Astrup'] < 50.) & (data['Chloor Astrup'] > 200.)] = np.nan
  data['Cl (onv.ISE) (bloed)'][(data['Cl (onv.ISE) (bloed)'] < 50.) & (data['Cl (onv.ISE) (bloed)'] > 200.)] = np.nan

  #make nans of all negative vals
  data[all_cols] = data[all_cols].applymap(lambda x: np.nan if x < 0 else x)
  return data


def remove_outliers_action(data):

  #delete outliers
  outliers = data.reset_index() #return to single index

  cols = ['Noradrenaline (Norepinefrine)', 'NaCl 0,45%/Glucose 2,5%']

  #select outlier cols
  data['Noradrenaline (Norepinefrine)'][data['Noradrenaline (Norepinefrine)'] > 10.] = np.nan
  data['NaCl 0,45%/Glucose 2,5%'][data['NaCl 0,45%/Glucose 2,5%'] > 500.] = np.nan
  

  data = data[cols].applymap(lambda x: np.nan if x < 0 else x)

  return data

def get_4h(data):
  #per patient, average the values in 4h timeslots

  data = data.sort_values('time')
  res = data.groupby([pd.Grouper('admissionid'),
                        pd.Grouper(key ='time', freq='4H')
                              
  ]).mean()

  return res

def get_aki(data):
  #per patient, take the first value in 4h timeslots

  data = data.sort_values('time')
  aki = data.groupby([pd.Grouper('admissionid'),
                        pd.Grouper(key ='time', freq='4H')
                              
    ]).first()

  return aki

def get_4h_urine(data):
  #per patient, average the values in 4h timeslots

  data = data.sort_values('time')
  res = data.groupby([pd.Grouper('admissionid'),
                        pd.Grouper(key ='time', freq='4H')
                              
  ]).sum()

  return res

def aggregate_col(data, colname):

  #create new columns with cumulative count for consecutive nans
  data['nancount'] = np.zeros(len(data))
  data['nancount'] = data[colname].groupby((data[colname].notnull()).cumsum()).cumcount()
                           
  #manually set first row to 1 if nan since this is excluded in cumsum/count from line before
  for i, v in enumerate(data[colname]):
    if i == 0:
      if np.isnan(v):
        data["nancount"] += 1
  
  #and all other ones +1, except non-nan values
  data["nancount"][data[colname] == np.nan] += 1

  #set non-null values as 0 for nancount
  data["nancount"] = np.where(~data[colname].isnull(), 0, data["nancount"])

  #if value is not nan, then use previous value + 1 to get total cumulative nan count including the non-nan value
  #this is the value we want to divide through to get the right value per hour

  group_val = pd.DataFrame(data[colname]).reset_index()[colname]
  group_count = pd.DataFrame(data["nancount"]).reset_index()["nancount"]

  for i, v in enumerate(group_count):

    if v == 0: #where no null values
 
      if (i != 0): #first row: do nothing
        if group_val[i] != 0: #value 0: do not divide
          if group_count[i-1] != 0: #value before is not missing: do nothing
          
            group_val[i] = group_val[i] / (group_count[i-1]) #otherwise: divide through nancount of row before
  

  #and then fill backwards and return
  return group_val.bfill()

def process_variables(data):

  # Sum Urine
  urine_cols = ['UrineCAD', 'UrineSupraPubis', 'UrineUP', 'UrineSpontaan', 'UrineIncontinentie', 'UrineSplint Re', 'UrineSplint Li']
  data['Urine_summed'] = data[urine_cols].sum(axis=1)
  condition = (data['Urine_summed'] == 0.)
  data[condition]['Urine_summed'] = np.nan
  data = data.drop(columns=urine_cols)

  # Mean per patient (Natrium)
  nat_cols = ['Natrium (bloed)', 'Natrium', 'Natrium Astrup', 'Na (onv.ISE) (bloed)']
  data['Natrium_processed'] = np.nanmean(data[nat_cols].mask(data[nat_cols].eq(0)).values, axis=1)
  condition = (data['Natrium_processed'] == 0.)
  data[condition]['Natrium_processed'] = np.nan
  data = data.drop(columns=nat_cols)

  # Group Mean Blood Pressure
  # If ABP I is available, use that one. Otherwise, use ABP II. If both are not available, set as NaN
  abp_cols = ['Niet invasieve bloeddruk gemiddeld', 'ABP gemiddeld II', 'ABP gemiddeld']
  data['ABP_processed'] = data['ABP gemiddeld']
  condition_1 = (data['ABP_processed'] == np.nan)
  condition_2 = (data['ABP_processed'] == 0.)
  condition = np.logical_or(condition_1, condition_2)
  data[condition]['ABP_processed'] = data[condition]['ABP gemiddeld II']
  condition_1 = (data['ABP_processed'] == np.nan)
  condition_2 = (data['ABP_processed'] == 0.)
  condition = np.logical_or(condition_1, condition_2)
  data[condition]['ABP_processed'] = data[condition]['Niet invasieve bloeddruk gemiddeld']
  condition = (data['ABP_processed'] == 0.)
  data[condition]['ABP_processed'] = np.nan
  data = data.drop(columns=abp_cols)

  # Mean per patient (Kalium)
  kal_cols = ['Kalium (bloed)', 'Kalium Astrup', 'Kalium', 'K (onv.ISE) (bloed)']
  data['Kalium_processed'] = np.nanmean(data[kal_cols].mask(data[kal_cols].eq(0)).values, axis=1)
  condition_1 = (data['Kalium_processed'] == np.nan)
  condition_2 = (data['Kalium_processed'] == 0.)
  condition = np.logical_or(condition_1, condition_2)
  data[condition]['Kalium_processed'] = np.nan
  data = data.drop(columns=kal_cols)

  # Mean per patient (Kreatinine)
  kreat_cols = ['Kreatinine', 'Kreatinine (bloed)', 'KREAT enzym. (bloed)']
  data['Kreatinine_processed'] = np.nanmean(data[kreat_cols].mask(data[kreat_cols].eq(0)).values, axis=1)
  condition_1 = (data['Kreatinine_processed'] == np.nan)
  condition_2 = (data['Kreatinine_processed'] == 0.)
  condition = np.logical_or(condition_1, condition_2)
  data[condition]['Kreatinine_processed'] = np.nan
  data = data.drop(columns=kreat_cols)

  # Mean per patient (Bicarbonate)
  bic_cols = ['HCO3', 'Act.HCO3 (bloed)']
  data['Bicarbonate_processed'] = np.nanmean(data[bic_cols].mask(data[bic_cols].eq(0)).values, axis=1)
  condition_1 = (data['Bicarbonate_processed'] == np.nan)
  condition_2 = (data['Bicarbonate_processed'] == 0.)
  condition = np.logical_or(condition_1, condition_2)
  data[condition]['Bicarbonate_processed'] = np.nan
  data = data.drop(columns=bic_cols)

  # Mean per patient (Chloor)
  cl_cols = ['Chloor (bloed)', 'Chloor Astrup', 'Chloor', 'Cl (onv.ISE) (bloed)']
  data['Chloor_processed'] = np.nanmean(data[cl_cols].mask(data[cl_cols].eq(0)).values, axis=1)
  condition_1 = (data['Chloor_processed'] == np.nan)
  condition_2 = (data['Chloor_processed'] == 0.)
  condition = np.logical_or(condition_1, condition_2)
  data[condition]['Chloor_processed'] = np.nan
  data = data.drop(columns=cl_cols)

  return data

def aggregate_all_cols(data, space, df):

  if space == 'state':

    cols_to_agg = ['time', 'admissionid', 'Natrium_processed', 'ABP_processed', 'Kalium_processed', 'Kreatinine_processed', 'Bicarbonate_processed', 'Chloor_processed']

    #group urine (sum)
    grouped = data.groupby('admissionid', as_index = False).apply(lambda x: aggregate_col(x, 'Urine_summed')).reset_index()['Urine_summed']
    data['Urine'] = list(grouped.head(len(grouped)))
    data = pd.DataFrame(data).reset_index()
    urine_aggr = get_4h_urine(data[['admissionid', 'time', 'Urine']])

    #group other variables (mean)
    data[cols_to_agg] = data[cols_to_agg].bfill()
    df_aggr = get_4h(data[cols_to_agg])

    #extract aki
    aki = get_aki(df[['admissionid','time','aki_stage']])

    #combine both aggregations
    combined = pd.concat([urine_aggr, df_aggr, aki], axis=1)

    return combined

  if space == 'action':

    data = data.reset_index()
    cols_to_agg = ['time', 'admissionid', 'Dobutamine (Dobutrex)',
                   'Adrenaline (Epinefrine)', 'Dopamine (Inotropin)',
                   'Noradrenaline (Norepinefrine)', 'NaCl 0,45%/Glucose 2,5%']
    data[cols_to_agg] = data[cols_to_agg].bfill()
    df_aggr = get_4h(data[cols_to_agg])

    return df_aggr

  else:

    print("ERROR INVALID SPACE TYPE: options for space: state, action")


def interpolate(data_agg):
  #interpolate null values
  return data_agg.interpolate(limit_direction='forward')

def transform_df(data: pd.DataFrame = None,
                 time_col: str = 'time',
                 bins: list = None,
                 bin_labels: list = None,
                 group_cols: list = ['admissionid', 'binn'],
                 agg_func: dict = None):
    """
    Transforms the input data from the AmsterdamUMCdb and return a dataframe with bins assigned to each record based on the time column
    :param data: dataframe with single timestamps as integers, patientid and values
    :param bins: list of bins to divide the timestamps in
    :param bin_labels: list of labels to name the bins with
    :param group_cols: list of column to group by, including the newly created 'binn'
    :param agg_func: dictionary of kwargs passed to the .agg() method
    """
    
    data['binn'] = pd.cut(data[time_col], bins=bins, labels=bin_labels)
    data = data[data[time_col]>=0]
    grouped_data = data.groupby(group_cols).agg(**agg_func).reset_index().sort_values(by=group_cols, ascending=True)
    
    return grouped_data


def transform_daterange(data: pd.DataFrame,
                        time_col: str = 'time',
                        infer_start_time: bool = True,
                        multi_source: bool = False,
                        multi_source_col: str = None,
                        start_time: str = 'start_time',
                        end_time: str = 'end_time',
                        time_unit: str = 'm',
                        value_col: str = 'value',
                        group_col: list = None,
                        fill_method: str = 'backfill',
                        fill_lim: int = 540
                        ):
    """
    Transform interval data with single timestamps to time range, calculate production, resample and backward fill
    :param data: dataframe with id, value and timestamp
    :param time_col: string representing the column name for the time of registration in a single timestamp dataframe
    :param infer_start_time: boolean representing whether the start time should be inferred from the previous record
    :param start_time: string representing the column name with the record start time
    :param end_time: string representing the column name with the record end time
    :param time_unit: interpret the integer timestamp as the given time unit and convert back to this unit at the end
    :param value_col: string representing the column name with the values of the measurements
    :param group_col: list representing the ids of patients and/or products
    :param fill_method: string to represent the method as used in pandas.series.fillna
    :param fill_lim: integer to represent the number of time units to be filled
    """
    
    if group_col is None:
        group_col = ['admissionid'] # PM: defining a list as default will keep alterations when rerunning the function
    
    # convert to datetime and set index to time column
    data[time_col] = pd.to_datetime(data[time_col], unit=time_unit)
    data[start_time] = pd.to_datetime(data[start_time], unit=time_unit)
    
    if infer_start_time:
        # get start time from previous record
        data['start_time'] = data.groupby(group_col)[time_col].shift(1)
        start_time = 'start_time'
        end_time = time_col
    else:
        # transform other columns to datetime if they exist and are still integer type, otherwise leave as is
        for t_col in [start_time, end_time]:
            if t_col in data:
                if pd.api.types.is_integer_dtype(data[start_time]):
                    data[t_col] = pd.to_datetime(data[t_col], unit=time_unit)
    
    # get time difference from start and end times   
    data['time_diff'] = (data[end_time] - data[start_time]) / np.timedelta64(1, time_unit)

    if multi_source:
        if multi_source_col is None:
            # give each record a unique id to group by in order to handle simultaneous records
            data['administrationid'] = range(data.shape[0])
            group_col += ['administrationid']
        else:
            group_col += [multi_source_col]
    
    # get production per time unit
    data['prod'] = data[value_col] / data['time_diff']
    
    # if start and end time are registered in the same record, create a new record with the other value as index
    if infer_start_time:
        data_merged = data.copy()
        data_merged.index = data_merged[time_col]
    else:
        data_end = data.copy()
        data.index = data.start_time
        data_end.index = data_end.stop_time
        data_merged = pd.concat([data, data_end]).sort_values(group_col + [start_time, end_time])
    
    # resample for each unit
    res = data_merged[group_col + ['prod', start_time, end_time]].groupby(group_col).resample('1T').mean().drop(group_col, axis=1, errors='ignore').reset_index().copy()
    
    # fill missing values
    res['prod_fill'] = res.groupby(group_col)['prod'].fillna(method=fill_method, limit=fill_lim) #9 hours
    
    # reset time column to integer values
    transform_time_col = {'s': 1, 'm': 60, 'h': 3600, 'd': 86_400}
    if infer_start_time:
        res[time_col] = (res[time_col].view(np.int64) / (transform_time_col.get(time_unit) * 1_000_000_000)).astype(int)
    else:
        if multi_source:
            level_col = 'level_2'
        else:
            level_col = 'level_1'
        res[time_col] = (res[level_col].view(np.int64) / (transform_time_col.get(time_unit) * 1_000_000_000)).astype(int)
    
    return res
    

def get_demograhics(data, df, admissionid):
  # Get gender
  gender = df['gender'][df['admissionid'] == admissionid].head(1).item()
  if gender == 'Man':
    gender = 0
  elif gender == 'Vrouw':
    gender = 1
  else:
    gender = 2

  # Get Age
  age = df['agegroup'][df['admissionid'] == admissionid].head(1).item()

  # Get Weight
  weight = df['weightgroup'][df['admissionid'] == admissionid].head(1).item()

  # Get Height
  height = df['heightgroup'][df['admissionid'] == admissionid].head(1).item()

  # Get PatientID
  patientID = df['patientid'][df['admissionid'] == admissionid].head(1).item()

  # Get Date of Death
  death = df['dateofdeath'][df['admissionid'] == admissionid].head(1).item()
  if (death == np.nan) or (pd.to_datetime(death, unit='ms', origin = 'unix') > data['time'][data['admissionid'] == admissionid].max()):
    death = 0
  else:
    death = 1

  return gender, age, weight, height, patientID, death


def complete_state(data, df):
  data = data.sort_values(by=['admissionid', 'time']).reset_index()
  genders = []
  ages = []
  weights = []
  heights = []
  patientids = []
  deaths = []

  # Get demographics for each admission id
  for x in data['admissionid']:
    gender, age, weight, height, patientID, death = get_demograhics(data, df, x)
    genders.append(gender)
    ages.append(age)
    weights.append(weight)
    heights.append(height)
    patientids.append(patientID)
    deaths.append(death)

  data['gender'] = genders
  data['agegroup'] = ages
  data['weightgroup'] = weights
  data['heightgroup'] = heights
  data['patientid'] = patientids
  data['death'] = deaths

  # Transform categorical variables into numerical
  categories = {"agegroup": {"18-39": 1, "40-49": 2, "50-59": 3, "60-69":4, "70-79":5, "80+":6}, 
          "weightgroup": {'59-': 1,'60-69': 2, '80-89': 3, '70-79': 4, '90-99': 5, '100-109': 6, '110+': 7}, 
          "heightgroup": {'159-': 1, '160-169': 2, '170-179': 3, '180-189': 4, '190+': 5}}
  data = data.replace(categories)
  
  return data


def process_statespace(data):
  data['time'] = pd.to_datetime(data['time'], unit='m', origin = 'unix')
  grouped = to_cols(data)
  grouped = remove_outliers(grouped)
  print(grouped.isnull().sum())
  data_sum = process_variables(grouped)
  print(data_sum.isnull().sum())
  data_agg = aggregate_all_cols(data_sum, space="state", df=data)
  print(data_agg.isnull().sum())
  data_agg = complete_state(data_agg, data)
  #data_filled = interpolate(data_agg)
  return data_agg.reset_index()

  
def process_actionspace(data):
  # data['time'] = pd.to_datetime(data['stop'] - data['start'], unit='ms')
  # data = data.drop(columns = ['start', 'stop'])
  # data['time'] = pd.to_datetime(data['time'], unit='ms', origin = 'unix')
  # grouped = to_cols_action(data)
  # #grouped = remove_outliers_action(grouped)
  # data_agg = aggregate_all_cols(grouped, space="action")
  # #data_filled = interpolate(data_agg)

  # Extract Fluids and Vasopressors
  fluids = data.loc[~data['itemid'].isin([7179,7178,6818,7229])]
  vasop = data.loc[data['itemid'].isin([7179,7178,6818,7229])]
  
  # Perform Aggregation
  df_aggr_fluids = transform_df(data=transform_daterange(fluids[['admissionid',
                                                                 'fluidin',
                                                                 'start_time',
                                                                 'stop_time']].sort_values(['admissionid', 'start_time']).copy(),
                                                     time_col = 'stop_time',
                                                     infer_start_time=False,
                                                     multi_source=False,
                                                     start_time = 'start_time',
                                                     end_time = 'stop_time',
                                                     value_col = 'fluidin',
                                                     group_col = ['admissionid']),
                                 time_col='stop_time',
                                 bins=range(0, 1216*60, 4*60),
                                 bin_labels=range(0, 1212*60, 4*60),
                                 group_cols=['admissionid', 'binn'],
                                 agg_func={'fluid_sum': ('prod_fill', 'sum')})
  
  df_aggr_vasops = transform_df(data=transform_daterange(vasop[['admissionid',
                                                                 'fluidin',
                                                                 'start_time',
                                                                 'stop_time']].sort_values(['admissionid', 'start_time']).copy(),
                                                     time_col = 'stop_time',
                                                     infer_start_time=False,
                                                     multi_source=False,
                                                     start_time = 'start_time',
                                                     end_time = 'stop_time',
                                                     value_col = 'fluidin',
                                                     group_col = ['admissionid']),
                                 time_col='stop_time',
                                 bins=range(0, 1216*60, 4*60),
                                 bin_labels=range(0, 1212*60, 4*60),
                                 group_cols=['admissionid', 'binn'],
                                 agg_func={'vasops_sum': ('prod_fill', 'sum')})
    
  df_aggr_fluids['fluid_sum'] = df_aggr_fluids['fluid_sum'].fillna(0)
  df_aggr_vasops['vasops_sum'] = df_aggr_vasops['vasops_sum'].fillna(0)

  df_aggr = pd.merge(df_aggr_fluids, df_aggr_vasops, how='outer', on=['admissionid', 'binn'])
  df_aggr['vasops_sum'] = df_aggr['vasops_sum'].fillna(0)

  df_aggr['time'] = pd.to_datetime(df_aggr['binn'].astype(float), unit='m', origin = 'unix')

  return df_aggr

## 3. Import Dataframes and Perform Aggregation

In [46]:
pd.set_option('display.max_rows', 100)

#some patients only get fluids, no vasopressor, is it possible?
#some patients do not receive either for quite some time, is it possible?# 


#action[action['admissionid'] == 23488]
action[action.admissionid == 23545]
#action.tail(100)

,admissionid,binn,fluid_sum,vasops_sum,time
859005,23545,0,1008.083333,NaN,1970-01-01 00:00:00
859006,23545,240,9223.109244,NaN,1970-01-01 04:00:00
859007,23545,480,24000.000000,NaN,1970-01-01 08:00:00
859008,23545,720,3428.174207,NaN,1970-01-01 12:00:00
859009,23545,960,3565.988316,NaN,1970-01-01 16:00:00
...,...,...,...,...,...
859303,23545,71520,0.000000,NaN,1970-02-19 16:00:00
859304,23545,71760,0.000000,NaN,1970-02-19 20:00:00
859305,23545,72000,0.000000,NaN,1970-02-20 00:00:00
859306,23545,72240,0.000000,NaN,1970-02-20 04:00:00


In [51]:
statespace = pd.read_csv('final_state_space.csv')
actionspace = pd.read_csv('final_action_space.csv')

state = process_statespace(statespace)
action = process_actionspace(actionspace)

item
admissionid                                0
time                                       0
ABP gemiddeld                          55772
ABP gemiddeld II                      209452
Act.HCO3 (bloed)                      157704
Chloor                                209328
Chloor (bloed)                        190565
Chloor Astrup                         209206
Cl (onv.ISE) (bloed)                  188981
HCO3                                  206967
K (onv.ISE) (bloed)                   168018
KREAT enzym. (bloed)                  209543
Kalium                                208723
Kalium (bloed)                        193923
Kalium Astrup                         207543
Kreatinine                            208880
Kreatinine (bloed)                    196228
Na (onv.ISE) (bloed)                  169373
Natrium                               208725
Natrium (bloed)                       193561
Natrium Astrup                        207570
Nefrodrain li Uit                     209072
Nefro

In [52]:
#merge datasets --> left merge on actionspace, as states without actions are not useful for our model
aggregated = action.merge(state, on=["admissionid", "time"], how="left")

In [54]:
print(len(action))
print(len(state))
print(len(aggregated))

859308
43894
859308


In [56]:
state.admissionid.nunique()

3135

In [57]:
action.admissionid.nunique()

2836

In [58]:
#all the null values are patients that are not in the state space and only in the action space --> we cannot use them so they are dropped
print('Dropped values:', aggregated.isnull().sum())
aggregated = aggregated.dropna()

Dropped values: admissionid                   0
binn                          0
fluid_sum                     0
vasops_sum                    0
time                          0
index                    820548
Urine                    820548
Natrium_processed        820548
ABP_processed            820548
Kalium_processed         820548
Kreatinine_processed     820552
Bicarbonate_processed    820548
Chloor_processed         820563
aki_stage                820548
gender                   820548
agegroup                 820548
weightgroup              822257
heightgroup              823596
patientid                820548
death                    820548
dtype: int64


In [59]:
aggregated.admissionid.nunique()

2582

In [61]:
aggregated.head()

,admissionid,binn,fluid_sum,vasops_sum,time,index,Urine,Natrium_processed,ABP_processed,Kalium_processed,Kreatinine_processed,Bicarbonate_processed,Chloor_processed,aki_stage,gender,agegroup,weightgroup,heightgroup,patientid,death
0,11,0,1564.114410,127.987797,1970-01-01 00:00:00,0.0,0.0,134.6,64.00,4.120,342.2,17.779999,107.20,0.0,0.0,4.0,3.0,4.0,11.0,0.0
1,11,240,1081.298935,189.248434,1970-01-01 04:00:00,1.0,0.0,134.8,72.40,4.600,337.0,14.580000,108.40,0.0,0.0,4.0,3.0,4.0,11.0,0.0
2,11,480,564.661661,198.434211,1970-01-01 08:00:00,2.0,0.0,136.6,71.20,4.560,316.0,17.200000,107.00,0.0,0.0,4.0,3.0,4.0,11.0,0.0
3,11,720,4650.132862,155.536134,1970-01-01 12:00:00,3.0,32.0,133.0,66.75,4.775,302.0,15.700000,108.25,0.0,0.0,4.0,3.0,4.0,11.0,0.0
4,11,960,1158.447309,160.000000,1970-01-01 16:00:00,4.0,20.0,135.2,63.80,4.580,302.0,18.820001,105.60,0.0,0.0,4.0,3.0,4.0,11.0,0.0


In [62]:
aggregated.head(100)

,admissionid,binn,fluid_sum,vasops_sum,time,index,Urine,Natrium_processed,ABP_processed,Kalium_processed,Kreatinine_processed,Bicarbonate_processed,Chloor_processed,aki_stage,gender,agegroup,weightgroup,heightgroup,patientid,death
0,11,0,1564.114410,127.987797,1970-01-01 00:00:00,0.0,0.0,134.600000,64.000000,4.120000,342.200000,17.779999,107.200000,0.0,0.0,4.0,3.0,4.0,11.0,0.0
1,11,240,1081.298935,189.248434,1970-01-01 04:00:00,1.0,0.0,134.800000,72.400000,4.600000,337.000000,14.580000,108.400000,0.0,0.0,4.0,3.0,4.0,11.0,0.0
2,11,480,564.661661,198.434211,1970-01-01 08:00:00,2.0,0.0,136.600000,71.200000,4.560000,316.000000,17.200000,107.000000,0.0,0.0,4.0,3.0,4.0,11.0,0.0
3,11,720,4650.132862,155.536134,1970-01-01 12:00:00,3.0,32.0,133.000000,66.750000,4.775000,302.000000,15.700000,108.250000,0.0,0.0,4.0,3.0,4.0,11.0,0.0
4,11,960,1158.447309,160.000000,1970-01-01 16:00:00,4.0,20.0,135.200000,63.800000,4.580000,302.000000,18.820001,105.600000,0.0,0.0,4.0,3.0,4.0,11.0,0.0
5,11,1200,563.910927,160.000000,1970-01-01 20:00:00,5.0,5.0,134.750000,61.500000,4.575000,286.000000,20.250000,106.500000,0.0,0.0,4.0,3.0,4.0,11.0,0.0
6,11,1440,6264.947415,156.162600,1970-01-02 00:00:00,6.0,0.0,134.000000,62.714286,4.485714,278.714286,17.657143,108.428571,0.0,0.0,4.0,3.0,4.0,11.0,0.0
7,11,1680,1752.301233,169.029761,1970-01-02 04:00:00,7.0,0.0,134.166667,73.500000,4.316667,269.000000,16.066667,109.000000,0.0,0.0,4.0,3.0,4.0,11.0,0.0
8,11,1920,1660.789216,161.846528,1970-01-02 08:00:00,8.0,0.0,136.666667,62.000000,5.033333,159.833333,13.816667,110.000000,0.0,0.0,4.0,3.0,4.0,11.0,0.0
606,25,0,463.012936,0.033333,1970-01-01 00:00:00,27.0,150.0,125.400000,65.600000,5.680000,222.800000,25.700000,89.400000,0.0,1.0,5.0,3.0,2.0,24.0,1.0


## 4. Save Aggregated Dataframe on Drive

In [ ]:
aggregated.to_csv('aggregated.csv')

## 5. Plot Statistics

In [ ]:
# Plot some stats, df with 1 row per patient for demographic stats
stats = state.drop_duplicates(subset=['admissionid'], keep='first')